In [ ]:
!pip install flask_wtf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 4.6 MB/s 


In [ ]:
from google.colab import auth
auth.authenticate_user()

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from PIL import Image
import pickle
from flask import Flask, request, jsonify, render_template
from form import ImageForm

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/mushroom_classification/models/inat/TPU_test_1')
label_dict = pickle.load(open('drive/MyDrive/mushroom_classification/dataframes/labels_inat.pkl', 'rb'))

In [ ]:
!gsutil -m cp -r gs://mushy_class/val .

Streaming output truncated to the last 5000 lines.
Copying gs://mushy_class/val/05634_Fuscoporia_gilva/d56a5c34-a739-4f1e-9200-64db685d2ad1.jpg...
Copying gs://mushy_class/val/05634_Fuscoporia_gilva/fdebc5c8-bb75-41a1-9d9e-7dcd07362202.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/007ee79f-9181-40cd-a600-c3fe80848577.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/1c955ae3-2808-409b-8646-d621a0535d13.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/3803ad82-9fe5-4d2c-9a0d-1d12a7729171.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/3d036ed2-63fd-479a-952f-afae1187120c.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/62c56adf-a4d3-4b6a-8da8-c3a60cea2810.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/77621cc0-2d91-47a0-8415-872b46037f60.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/7a4d7f0e-9b7a-4b85-8536-6a76c0761201.jpg...
Copying gs://mushy_class/val/05635_Inonotus_obliquus/cda4ab3f-b015-430a-aaf5-5

In [ ]:
def classify(file):
    img = tf.io.read_file(file)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [512, 512])
    img = tf.expand_dims(img, 0)

    preds = model.predict(img)
    sorted_preds = preds[0].argsort()[::-1][:100]

    predictions = {label_dict[pred]: round(preds[0][pred] * 100, 4) for pred in sorted_preds[:10]}

    return predictions

In [ ]:
preds = classify('/content/val/05466_Hypomyces_lactifluorum/971a4fdf-8de9-44ac-a5cd-cb5daf5eea82.jpg')